## A Case Study - Power Consumption Optimization
We can apply the basic ideas used in the linear optimization to a problem of power consumption optimization. In energy management, decreasing the peak power demand is an important task. We can optimize the schedules of appliances (for a household for example) such that maximum power demand is minimized.


#### Objective
The objective definition is rather straight forward. If we let $P_i^t$ denote the power consumption for the appliance $i$ at time $t$, then the total power consumption at any time $t$ is $P^t = \sum_i P_i^t$. We can define the maximum power demand $P_{max} = \max P^t$. Then the objective function is then simply:
$$
\min P_{max}
$$
#### Variables
For this problem we can assume the variables are the power consumption of an appliance at any time t. We will denote this with $P_i^t$. For $N$ appliances and $T$ time steps, there will be $N \times T$ variables. For programming convenience, we will assume that all the appliances are either on or off, consuming either $0$ or $P_i$ amount of power. We can introduce binary integer variables $x_i^t$ to denote the state of the appliance $i$ at time $t$.
#### Constraints
We introduced both $p_i^t$ and $x_i^t$. First type of constraint arises due to their dependency.
$$
P_i^t = P_i x_i^t \hspace{1cm} \forall i, t
$$
Another type of constraint is the total energy consumption for appliances. We want the schedules of the applianes to be optimized, but the total amount of runtime (total amount of energy consumption) should not change.
$$
\sum_t P_i^t = E_i \hspace{1cm} \forall i
$$

In [2]:
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpDot, lpSum
from utilities import print_result, print_power_values

In [3]:
appliances = list(range(1, 4))
time_steps = list(range(1, 11))
p = LpVariable.dict('P', (appliances, time_steps), lowBound=0, cat='Continuous')
x = LpVariable.dict('x', (appliances, time_steps), lowBound=0, upBound=1, cat='Integer')

# Rated power for appliances
power_values = {
    1: 100,
    2: 50,
    3: 75
}

# Number of time steps each appliance is required to run
runtimes = {
    1: 5,
    2: 3,
    3: 4
}

# introduce max power as a new variable
p_max = LpVariable('P_max', lowBound=0, cat='Continuous')

problem = LpProblem('Minimize maximum power demand', LpMinimize)

for i in appliances:
    for t in time_steps:
        # constraint P_i_t = x_i_t * P_i
        problem += p[(i, t)] == x[(i, t)] * power_values[i]
    
    # total runtime constraint for each appliance
    problem += lpSum([x[(i, t)] for t in time_steps]) == runtimes[i]
    
for t in time_steps:
    # for any time step t total power consumption <= p_max 
    problem += lpSum([p[(i, t)] for i in appliances]) <= p_max

problem += p_max

problem.solve()
print_power_values(problem, p, appliances, time_steps)

Time	App1	App2	App3
1	0	0	75
2	0	50	75
3	0	50	75
4	100	0	0
5	100	0	0
6	100	0	0
7	100	0	0
8	0	50	75
9	100	0	0
10	0	0	0
